In [1]:
#da https://chatgpt.com/share/6959abcc-287c-8013-a2a7-8b7190ae7fd2

#setup e configurazione

import os, re, json, time, math
from difflib import SequenceMatcher
from urllib.parse import quote_plus

import pandas as pd
import requests

INPUT_CSV  = "biblio_longform_all_records.csv"
OUTPUT_CSV = "biblio_longform_all_records_enriched.csv"
PROGRESS_JSON = "biblio_enrich_progress.json"

# --- Networking ---
SESSION = requests.Session()
SESSION.headers.update({
    "User-Agent": "biblio-enricher/1.0 (academic research; contact: pietro.terna@unito.it)"
})
TIMEOUT = 30
SLEEP_BETWEEN_CALLS = 0.25  # prudenziale per non martellare i servizi

# --- Matching ---
TOPK_PER_SOURCE = 5

def _sleep():
    time.sleep(SLEEP_BETWEEN_CALLS)

def norm(s: str) -> str:
    if s is None:
        return ""
    s = str(s).strip().lower()
    s = re.sub(r"\s+", " ", s)
    s = re.sub(r"[“”\"'’`]", "", s)
    s = re.sub(r"[^0-9a-zàèéìòùçäëïöüßñ \-:/]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

def sim(a: str, b: str) -> float:
    a, b = norm(a), norm(b)
    if not a or not b:
        return 0.0
    return SequenceMatcher(None, a, b).ratio()

def extract_year(s: str):
    if not s:
        return None
    m = re.search(r"(1[5-9]\d{2}|20\d{2})", str(s))
    return int(m.group(1)) if m else None

def safe_get(d, *keys, default=None):
    cur = d
    for k in keys:
        if isinstance(cur, dict) and k in cur:
            cur = cur[k]
        else:
            return default
    return cur

def load_progress():
    if os.path.exists(PROGRESS_JSON):
        with open(PROGRESS_JSON, "r", encoding="utf-8") as f:
            return json.load(f)
    return {"done_n_scheda": [], "last_index": 0}

def save_progress(prog):
    with open(PROGRESS_JSON, "w", encoding="utf-8") as f:
        json.dump(prog, f, ensure_ascii=False, indent=2)

print("Setup OK.")


Setup OK.


In [2]:
#lettura longform e raggruppamento “libro per libro”

df = pd.read_csv(INPUT_CSV)

# Sanity check
assert set(df.columns) >= {"n_scheda","campo","valore"}, "CSV non nel formato atteso (n_scheda,campo,valore)."

# Raggruppo per scheda (libro)
groups = list(df.groupby("n_scheda", sort=True))
n_books = len(groups)

print("Righe:", len(df), " | Libri (n_scheda distinti):", n_books)
print("Esempio campi:", sorted(df["campo"].unique())[:20])


Righe: 11477  | Libri (n_scheda distinti): 1000
Esempio campi: ['autore', 'cod_isbn', 'cod_sez', 'collabor', 'collezion', 'data_pub', 'editore', 'edizione', 'formato', 'luogo_pub', 'n_s_sez', 'n_scheda', 'note_sp', 'pagine', 'soggetto', 'titolo']


In [3]:
#estrazione dati “di input” (titolo/autore/anno/ISBN ecc.)

def book_input_from_group(g: pd.DataFrame) -> dict:
    # prende la "migliore" occorrenza per campo
    # (se ci sono ripetizioni, tiene la prima non vuota)
    rec = {"n_scheda": int(g["n_scheda"].iloc[0])}

    def first_value(field):
        vals = g.loc[g["campo"] == field, "valore"].dropna().astype(str).tolist()
        vals = [v.strip() for v in vals if v.strip() and v.strip().lower() != "nan"]
        return vals[0] if vals else ""

    rec["autore"] = first_value("autore")
    rec["titolo"] = first_value("titolo")
    rec["editore"] = first_value("editore")
    rec["luogo_pub"] = first_value("luogo_pub")
    rec["data_pub"] = first_value("data_pub")
    rec["cod_isbn"] = first_value("cod_isbn")

    rec["year"] = extract_year(rec["data_pub"]) or extract_year(rec["titolo"])
    # query “compatta” per motori che accettano full text
    q_parts = [rec["titolo"], rec["autore"], str(rec["year"]) if rec["year"] else ""]
    rec["q"] = " ".join([p for p in q_parts if p]).strip()

    return rec

# preview di un libro
sample = book_input_from_group(groups[0][1])
sample


{'n_scheda': 41186,
 'autore': 'CARINGTON, WHATELY',
 'titolo': '',
 'editore': 'ASTROLABIO',
 'luogo_pub': 'ROMA',
 'data_pub': 'COPYRIGHT 1972',
 'cod_isbn': '',
 'year': 1972,
 'q': 'CARINGTON, WHATELY 1972'}

In [4]:
#client OPAC SBN (opacmobilegw)

def sbn_search(q: str, rows=TOPK_PER_SOURCE):
    # endpoint app mobile (non ufficiale)
    url = f"https://opac.sbn.it/opacmobilegw/search.json?any={quote_plus(q)}&type=0&start=0&rows={rows}"
    r = SESSION.get(url, timeout=TIMEOUT)
    _sleep()
    r.raise_for_status()
    return r.json()

def sbn_full(bid: str):
    # bid in forma "IT\\ICCU\\..." nella search; in URL bisogna passarlo con backslash singoli
    url = f"https://opac.sbn.it/opacmobilegw/full.json?bid={quote_plus(bid)}"
    r = SESSION.get(url, timeout=TIMEOUT)
    _sleep()
    r.raise_for_status()
    return r.json()

def sbn_candidates(book: dict):
    if not book["q"]:
        return []
    try:
        js = sbn_search(book["q"])
    except Exception:
        return []

    hits = js.get("records") or js.get("record") or js.get("results") or js.get("docs") or js.get("elenco") or js.get("items")
    # in molte risposte reali l'elenco è in chiave "records"; ma manteniamo fallback
    if hits is None:
        hits = js.get("records", [])

    if not isinstance(hits, list):
        hits = []

    cands = []
    for h in hits[:TOPK_PER_SOURCE]:
        title = h.get("titolo") or ""
        auth  = h.get("autorePrincipale") or ""
        pub   = h.get("pubblicazione") or ""
        bid   = h.get("codiceIdentificativo") or ""

        score = 0.70 * sim(book["titolo"], title) + 0.30 * sim(book["autore"], auth)
        by = extract_year(pub) or None
        if book.get("year") and by and abs(book["year"] - by) <= 1:
            score += 0.05

        cands.append({"source":"OPAC SBN", "score":score, "title":title, "author":auth, "pub":pub, "bid":bid, "raw":h})

    # arricchisco col full.json solo per i migliori 2 (per ridurre traffico)
    cands = sorted(cands, key=lambda x: x["score"], reverse=True)
    for c in cands[:2]:
        if c.get("bid"):
            try:
                full = sbn_full(c["bid"])
                c["full"] = full
            except Exception:
                c["full"] = None
    return cands


In [5]:
#Open Library

def ol_search(title: str, author: str, rows=TOPK_PER_SOURCE):
    # Search API ufficiale
    params = {"title": title or "", "author": author or "", "limit": rows}
    r = SESSION.get("https://openlibrary.org/search.json", params=params, timeout=TIMEOUT)
    _sleep()
    r.raise_for_status()
    return r.json()

def ol_candidates(book: dict):
    if not (book["titolo"] or book["autore"]):
        return []
    try:
        js = ol_search(book["titolo"], book["autore"])
    except Exception:
        return []
    docs = js.get("docs", [])[:TOPK_PER_SOURCE]
    cands = []
    for d in docs:
        title = d.get("title") or ""
        auth  = (d.get("author_name") or [""])[0]
        year  = d.get("first_publish_year")
        score = 0.70 * sim(book["titolo"], title) + 0.30 * sim(book["autore"], auth)
        if book.get("year") and year and abs(book["year"] - int(year)) <= 1:
            score += 0.05
        cands.append({"source":"Open Library", "score":score, "title":title, "author":auth, "year":year, "raw":d})
    return sorted(cands, key=lambda x: x["score"], reverse=True)


In [6]:
#Library of Congress (loc.gov JSON)

def loc_search(q: str, rows=TOPK_PER_SOURCE):
    # endpoint "books" su loc.gov
    params = {"fo": "json", "q": q, "c": rows}
    r = SESSION.get("https://www.loc.gov/books/", params=params, timeout=TIMEOUT)
    _sleep()
    r.raise_for_status()
    return r.json()

def loc_candidates(book: dict):
    if not book["q"]:
        return []
    try:
        js = loc_search(book["q"])
    except Exception:
        return []
    results = js.get("results", [])[:TOPK_PER_SOURCE]
    cands = []
    for it in results:
        title = it.get("title") or ""
        # loc.gov spesso mette autore in 'contributor' o 'creator'
        auth_list = it.get("contributor") or it.get("creator") or []
        auth = auth_list[0] if isinstance(auth_list, list) and auth_list else ""
        date = it.get("date") or ""
        year = extract_year(date)

        score = 0.70 * sim(book["titolo"], title) + 0.30 * sim(book["autore"], auth)
        if book.get("year") and year and abs(book["year"] - year) <= 1:
            score += 0.05

        cands.append({"source":"Library of Congress", "score":score, "title":title, "author":auth, "date":date, "id":it.get("id"), "raw":it})
    return sorted(cands, key=lambda x: x["score"], reverse=True)


In [7]:
#Internet Archive (advancedsearch + metadata)

def ia_advanced_search(title: str, author: str, rows=TOPK_PER_SOURCE):
    # query in sintassi IA (Elastic/Lucene-like)
    # limitiamo ai testi (dove possibile) per ridurre rumore
    q = []
    if title:
        q.append(f'title:("{title}")')
    if author:
        q.append(f'creator:("{author}")')
    q.append('(mediatype:texts OR collection:opensource_texts)')
    query = " AND ".join(q)

    params = {
        "q": query,
        "fl[]": ["identifier", "title", "creator", "date", "publisher", "language"],
        "rows": rows,
        "page": 1,
        "output": "json"
    }
    r = SESSION.get("https://archive.org/advancedsearch.php", params=params, timeout=TIMEOUT)
    _sleep()
    r.raise_for_status()
    return r.json()

def ia_metadata(identifier: str):
    r = SESSION.get(f"https://archive.org/metadata/{identifier}", timeout=TIMEOUT)
    _sleep()
    r.raise_for_status()
    return r.json()

def ia_candidates(book: dict):
    if not (book["titolo"] or book["autore"]):
        return []
    try:
        js = ia_advanced_search(book["titolo"], book["autore"])
    except Exception:
        return []

    docs = safe_get(js, "response", "docs", default=[])[:TOPK_PER_SOURCE]
    cands = []
    for d in docs:
        title = d.get("title") or ""
        creator = d.get("creator")
        auth = creator[0] if isinstance(creator, list) and creator else (creator or "")
        year = extract_year(d.get("date"))

        score = 0.70 * sim(book["titolo"], title) + 0.30 * sim(book["autore"], auth)
        if book.get("year") and year and abs(book["year"] - year) <= 1:
            score += 0.05

        cands.append({"source":"Internet Archive", "score":score, "title":title, "author":auth, "year":year,
                      "identifier": d.get("identifier"), "raw":d})

    cands = sorted(cands, key=lambda x: x["score"], reverse=True)

    # metadata completa solo per il migliore
    if cands and cands[0].get("identifier"):
        try:
            cands[0]["full"] = ia_metadata(cands[0]["identifier"])
        except Exception:
            cands[0]["full"] = None

    return cands


In [8]:
#normalizzazione output “campi standard” (ext_*)

STANDARD_FIELDS = [
    "ext_source",
    "ext_match_score",
    "ext_title",
    "ext_author",
    "ext_year",
    "ext_publisher",
    "ext_place",
    "ext_language",
    "ext_physical_desc",
    "ext_identifiers",
    "ext_notes",
    "ext_url"
]

def pack_identifiers(d: dict) -> str:
    # stringa JSON compatta, così non perdiamo struttura
    return json.dumps(d, ensure_ascii=False)

def from_sbn(best: dict) -> dict:
    full = best.get("full") or {}
    out = {
        "ext_source": "OPAC SBN",
        "ext_match_score": f"{best.get('score',0):.3f}",
        "ext_title": full.get("titolo") or best.get("title") or "",
        "ext_author": full.get("autorePrincipale") or best.get("author") or "",
        "ext_year": str(extract_year(full.get("pubblicazione")) or extract_year(best.get("pub")) or "") or "",
        "ext_publisher": "",  # spesso è dentro 'pubblicazione' non splittata
        "ext_place": "",
        "ext_language": full.get("linguaPubblicazione") or "",
        "ext_physical_desc": full.get("descrizioneFisica") or "",
        "ext_identifiers": pack_identifiers({"bid": best.get("bid")}),
        "ext_notes": " | ".join(full.get("note") or []) if isinstance(full.get("note"), list) else (full.get("note") or ""),
        "ext_url": f"https://opac.sbn.it/opacmobilegw/full.json?bid={quote_plus(best.get('bid',''))}" if best.get("bid") else ""
    }
    return out

def from_ol(best: dict) -> dict:
    d = best.get("raw") or {}
    identifiers = {}
    for k in ["isbn", "oclc", "lccn"]:
        if k in d and d[k]:
            identifiers[k] = d[k][:10] if isinstance(d[k], list) else d[k]
    out = {
        "ext_source": "Open Library",
        "ext_match_score": f"{best.get('score',0):.3f}",
        "ext_title": best.get("title") or "",
        "ext_author": best.get("author") or "",
        "ext_year": str(best.get("year") or ""),
        "ext_publisher": (d.get("publisher") or [""])[0] if isinstance(d.get("publisher"), list) else (d.get("publisher") or ""),
        "ext_place": (d.get("publish_place") or [""])[0] if isinstance(d.get("publish_place"), list) else (d.get("publish_place") or ""),
        "ext_language": (d.get("language") or [""])[0] if isinstance(d.get("language"), list) else (d.get("language") or ""),
        "ext_physical_desc": "",
        "ext_identifiers": pack_identifiers(identifiers),
        "ext_notes": "",
        "ext_url": f"https://openlibrary.org{d.get('key','')}" if d.get("key") else ""
    }
    return out

def from_loc(best: dict) -> dict:
    d = best.get("raw") or {}
    out = {
        "ext_source": "Library of Congress",
        "ext_match_score": f"{best.get('score',0):.3f}",
        "ext_title": best.get("title") or "",
        "ext_author": best.get("author") or "",
        "ext_year": str(extract_year(best.get("date") or "") or ""),
        "ext_publisher": "",  # spesso richiede item detail; qui lo teniamo vuoto
        "ext_place": "",
        "ext_language": "",
        "ext_physical_desc": "",
        "ext_identifiers": pack_identifiers({"loc_id": best.get("id")}),
        "ext_notes": "",
        "ext_url": best.get("id") or ""
    }
    return out

def from_ia(best: dict) -> dict:
    d = best.get("raw") or {}
    full = best.get("full") or {}
    md = full.get("metadata") or {}

    # md può avere liste o stringhe
    def md1(k):
        v = md.get(k, "")
        if isinstance(v, list):
            return v[0] if v else ""
        return v or ""

    identifiers = {"identifier": best.get("identifier")}
    out = {
        "ext_source": "Internet Archive",
        "ext_match_score": f"{best.get('score',0):.3f}",
        "ext_title": md1("title") or best.get("title") or "",
        "ext_author": md1("creator") or best.get("author") or "",
        "ext_year": str(extract_year(md1("date")) or best.get("year") or ""),
        "ext_publisher": md1("publisher"),
        "ext_place": md1("publishplace"),
        "ext_language": md1("language"),
        "ext_physical_desc": md1("description"),
        "ext_identifiers": pack_identifiers(identifiers),
        "ext_notes": md1("notes"),
        "ext_url": f"https://archive.org/details/{best.get('identifier')}" if best.get("identifier") else ""
    }
    return out

def choose_best_candidate(book: dict):
    # cascata: SBN -> OL -> LOC -> IA
    # ma se SBN non supera soglia minima, prosegui
    MIN_SCORE = 0.4 #0.55

    sbn = sbn_candidates(book)
    if sbn and sbn[0]["score"] >= MIN_SCORE:
        return sbn[0], from_sbn(sbn[0])

    ol = ol_candidates(book)
    if ol and ol[0]["score"] >= MIN_SCORE:
        return ol[0], from_ol(ol[0])

    loc = loc_candidates(book)
    if loc and loc[0]["score"] >= MIN_SCORE:
        return loc[0], from_loc(loc[0])

    ia = ia_candidates(book)
    if ia and ia[0]["score"] >= MIN_SCORE:
        return ia[0], from_ia(ia[0])

    return None, {"ext_source":"NOT_FOUND", "ext_match_score":"0.000",
                  "ext_title":"", "ext_author":"", "ext_year":"",
                  "ext_publisher":"", "ext_place":"", "ext_language":"",
                  "ext_physical_desc":"", "ext_identifiers":pack_identifiers({}),
                  "ext_notes":"", "ext_url":""}


In [9]:
#loop “uno per uno”, con stampa del numero libro (1..N) e salvataggio incrementale

prog = load_progress()
done = set(prog.get("done_n_scheda", []))

out_rows = []
# Se esiste già un output, lo carico e ci appendo (ripartenza)
if os.path.exists(OUTPUT_CSV):
    existing = pd.read_csv(OUTPUT_CSV)
    out_rows = existing.to_dict(orient="records")
    print("Ripartenza: output esistente caricato:", len(existing), "righe")
else:
    out_rows = df.to_dict(orient="records")

# indicizzazione rapida: per ogni n_scheda, quante righe originali già presenti in out_rows?
# (out_rows contiene già originali; noi aggiungiamo solo le righe ext_*; per evitare doppioni controlliamo)
already_ext = set()
for r in out_rows:
    if str(r.get("campo","")).startswith("ext_"):
        already_ext.add((int(r["n_scheda"]), r["campo"]))

for i, (n_scheda, g) in enumerate(groups, start=1):
    CURRENT_BOOK_NUMBER = i
    print(f"[{CURRENT_BOOK_NUMBER}/{n_books}] n_scheda={n_scheda}")

    if int(n_scheda) in done:
        # già processato: salto
        continue

    book = book_input_from_group(g)
    cand, ext = choose_best_candidate(book)

    # aggiungo righe longform ext_* in fondo alla scheda (append al dataset)
    for k in STANDARD_FIELDS:
        if (int(n_scheda), k) in already_ext:
            continue
        out_rows.append({"n_scheda": int(n_scheda), "campo": k, "valore": ext.get(k, "")})
        already_ext.add((int(n_scheda), k))

    # checkpoint
    done.add(int(n_scheda))
    prog["done_n_scheda"] = sorted(list(done))
    prog["last_index"] = i
    save_progress(prog)

    # scrittura su disco ogni 5 libri (riduci rischio di perdere lavoro)
    if i % 5 == 0:
        pd.DataFrame(out_rows).to_csv(OUTPUT_CSV, index=False)
        print("Checkpoint scritto:", OUTPUT_CSV)

# scrittura finale
pd.DataFrame(out_rows).to_csv(OUTPUT_CSV, index=False)
print("Completato. Output:", OUTPUT_CSV)


Ripartenza: output esistente caricato: 12377 righe
[1/1000] n_scheda=41186
[2/1000] n_scheda=41187
[3/1000] n_scheda=41188
[4/1000] n_scheda=41189
[5/1000] n_scheda=41190
[6/1000] n_scheda=41191
[7/1000] n_scheda=41192
[8/1000] n_scheda=41193
[9/1000] n_scheda=41194
[10/1000] n_scheda=41195
[11/1000] n_scheda=41196
[12/1000] n_scheda=41197
[13/1000] n_scheda=41198
[14/1000] n_scheda=41199
[15/1000] n_scheda=41200
[16/1000] n_scheda=41201
[17/1000] n_scheda=41202
[18/1000] n_scheda=41203
[19/1000] n_scheda=41204
[20/1000] n_scheda=41205
[21/1000] n_scheda=41206
[22/1000] n_scheda=41207
[23/1000] n_scheda=41208
[24/1000] n_scheda=41209
[25/1000] n_scheda=41210
[26/1000] n_scheda=41211
[27/1000] n_scheda=41212
[28/1000] n_scheda=41213
[29/1000] n_scheda=41214
[30/1000] n_scheda=41215
[31/1000] n_scheda=41216
[32/1000] n_scheda=41217
[33/1000] n_scheda=41218
[34/1000] n_scheda=41219
[35/1000] n_scheda=41220
[36/1000] n_scheda=41221
[37/1000] n_scheda=41222
[38/1000] n_scheda=41223
[39/1000

In [10]:
#da longform a wide (con gestione duplicati)

import pandas as pd

ENRICHED_LONGFORM = "biblio_longform_all_records_enriched.csv"
OUTPUT_WIDE = "biblio_wide_records_enriched.csv"

df_long = pd.read_csv(ENRICHED_LONGFORM)
df_long["valore"] = df_long["valore"].fillna("").astype(str)

# Evita collisione: il campo "n_scheda" (in colonna 'campo') viene rinominato
df_long = df_long.copy()
df_long.loc[df_long["campo"] == "n_scheda", "campo"] = "n_scheda_field"

# Aggregazione (se per lo stesso campo ci sono più valori, li concatena)
agg = (df_long
       .groupby(["n_scheda", "campo"])["valore"]
       .agg(lambda s: " | ".join([v.strip() for v in s if isinstance(v, str) and v.strip()]))
       .reset_index()
      )

# Pivot wide
df_wide = agg.pivot(index="n_scheda", columns="campo", values="valore").reset_index()

# Riordino colonne (opzionale)
front = ["n_scheda", "titolo", "autore", "data_pub", "editore",
         "ext_source", "ext_match_score", "ext_url", "ext_identifiers", "n_scheda_field"]
cols = [c for c in front if c in df_wide.columns] + [c for c in df_wide.columns if c not in front]
df_wide = df_wide[cols]

df_wide.to_csv(OUTPUT_WIDE, index=False)
print("Creato:", OUTPUT_WIDE, "| righe:", len(df_wide), "| colonne:", len(df_wide.columns))


Creato: biblio_wide_records_enriched.csv | righe: 1000 | colonne: 29
